# Example of a Convolutional Neural Network (12-Multiclass) for Text Classification

We first include all the necesary imports.

In [ ]:
#Imports needed from pytorch
import torch
import torch.nn as nn

#Some built-in imports
import numpy as np
import matplotlib.pyplot as plt
import pickle
from os.path import join, isfile

#Imports from the repository
from data_processing import get_weight_matrix
from privacy_policies_dataset import PrivacyPoliciesDataset as PPD
from cnn import CNN

In [ ]:
def confusion_matrix(model, dataset, label, threshold):
    
    metrics = {}
    x = PPD.collate_data(dataset)[0]
    y_hat = model(x)
    y_hat = torch.ge(y_hat, threshold).double()
    y = dataset.labels_tensor[:,label].unsqueeze(1).double() 
    tp = (y * y_hat).sum()
    tn = ((1 - y) * (1 - y_hat)).sum()
    fp = (y_hat * (1 - y)).sum()
    fn = ((1 - y_hat) * y).sum()
    metrics['TP{}'.format(label)] = tp
    metrics['TN{}'.format(label)] = tn
    metrics['FP{}'.format(label)] = fp
    metrics['FN{}'.format(label)] = fn
    
    return metrics

def save_confusion_matrix(matrix, label, name, output_path):
    
    output = join(output_path, 'cm{}_{}.pkl'.format(label, name))
    
    with open(output, 'wb') as f:
        
        pickle.dump(matrix, f)
        
def load_confusion_matrix(path, label):
    
    matrix_path = join(path, 'cm{}.pkl'.format(label))
    
    with open(matrix_path, 'rb') as f:
        
        return pickle.load(f)

In [ ]:
def confusion_matrices(model, dataset, threshold):
    
    metrics = {}
    x = PPD.collate_data(dataset)[0]
    y_hat = model(x)
    y_hat = torch.ge(y_hat, threshold).double()
    y = dataset.labels_tensor.double() 
    tp = (y * y_hat).sum(0).numpy()
    tn = ((1 - y) * (1 - y_hat)).sum(0).numpy()
    fp = (y_hat * (1 - y)).sum(0).numpy()
    fn = ((1 - y_hat) * y).sum(0).numpy()
    metrics['TP'] = tp
    metrics['TN'] = tn
    metrics['FP'] = fp
    metrics['FN'] = fn
    
    return metrics

def confusion_matrices_change(metrics_05, metrics_best_t):
    
    labels = range(12)
    fig = plt.figure()
    fig.suptitle("metrics differences")
    tp_ax = fig.add_subplot(411)
    tn_ax = fig.add_subplot(412)
    fp_ax = fig.add_subplot(413)
    fn_ax = fig.add_subplot(414)
    
    tp_ax.plot(labels, metrics_05['TP'])
    tn_ax.plot(labels, metrics_05['TN'])
    fp_ax.plot(labels, metrics_05['FP'])
    fn_ax.plot(labels, metrics_05['FN'])
    
    tp_ax.plot(labels, metrics_05['TP'])
    tn_ax.plot(labels, metrics_05['TN'])
    fp_ax.plot(labels, metrics_05['FP'])
    fn_ax.plot(labels, metrics_05['FN'])
    
    plt.show()

In [ ]:
def get_metrics_t(model, training_dataset, validation_dataset):
    
    threshold_list = np.arange(0.0, 1, 0.01)
    
    eps = 1e-10
    
    f1_scores = []
    
    precisions = []
    
    recalls = []
    
    for t in threshold_list:
        
        cm = confusion_matrix(model, validation_dataset, label, t)
        
        tp = cm['TP{}'.format(label)]
        
        fp = cm['FP{}'.format(label)]
        
        fn = cm['FN{}'.format(label)]
        
        f1 = 2 * tp / (2 * tp + fn + fp + eps)
        
        precision = tp / (tp + fp + eps)
        
        recall = tp / (tp + fn + eps)
        
        f1_scores.append(f1)
        
        precisions.append(precision)
        
        recalls.append(recalls)
        
    return f1_scores, precisions, recalls

In [ ]:
def get_best_threshold(model, validation_dataset, label):
    
    eps = 1e-10
    
    threshold_list = np.arange(0.0, 1, 0.01)
    
    best_f1 = 0
    
    best_t = 0
    
    for t in threshold_list:
        
        cm = confusion_matrix(model, validation_dataset, label, t)
        
        tp = cm['TP{}'.format(label)]
        
        fp = cm['FP{}'.format(label)]
        
        fn = cm['FN{}'.format(label)]
        
        f1 = 2 * tp / (2 * tp + fn + fp + eps) 
        
        if f1 > best_f1:
            
            best_f1 = f1
            
            best_t = t
            
    return best_f1, best_t

## Data Loading

The first step is to load the previously split data into the three datasets: train, validation and text dataset.

In [ ]:
test_dataset = PPD.unpickle_dataset("datasets/test_dataset_label6.pkl")

train_dataset = PPD.unpickle_dataset("datasets/train_dataset_label6.pkl")

validation_dataset = PPD.unpickle_dataset("datasets/validation_dataset_label6.pkl")

In [ ]:
train_dataset.labels_stats()

We now load the weight matrix that will be used in the embedding layer. If the weight matrix hasn't been created yet we will need to use the data_processing.py to do so.

In [ ]:
weights_matrix = get_weight_matrix(300, "embeddings_data/FastTextMatrices/NoOOV", read = True)

Now we take a look to all the labels and select one of them to be trained

In [ ]:
labels = train_dataset.labels

for label, index in labels.items():
    
    print(str(index) + '. ' + label)

In [ ]:
label = int(raw_input("Insert label index: "))

for label_name, index in labels.items():
    
    if index == label:
        
        print(str(index) + '. ' + label_name)
        

## Training the CNN

The last step is to instatntiate a CNN with the desired parameters, load the embeddings and train the model.

In [ ]:
model = CNN(weights_matrix.shape[0], weights_matrix.shape[1], 200, [100], 1, [3], name = 'label{}_polisis'.format((label)))

model.load_pretrained_embeddings(weights_matrix)

In [ ]:
results = model.train_label(train_dataset, validation_dataset, label, epochs_num = 20, lr = 0.005, alpha = 0, momentum = 0.9)

epochs, train_losses, validation_losses = results

## Training all labels

In [ ]:
epochs_num = [60, 60, 150, 150, 70, 100, 150, 100, 70, 65, 80, 60]

lr = 0.001

best_thresholds = []

for label in range(12):    
    
    if epochs_num[label] == 100 or label in [0,1,2,3]:
        
        continue
    
    model = CNN(weights_matrix.shape[0], weights_matrix.shape[1], 200, [100, 25], 1, [3], 
                name = 'e{}_label{}_polisis'.format(epochs_num[label],label))

    model.load_pretrained_embeddings(weights_matrix)

    results = model.train_label(train_dataset, validation_dataset, label, epochs_num = epochs_num[label], lr = lr, 
                                alpha = 0, momentum = 0.9)

    epochs, train_losses, validation_losses = results

    plt.plot(epochs, train_losses, label = "train")

    plt.plot(epochs, validation_losses, label = "validation")

    plt.legend()

    plt.title("loss vs epoch")

    plt.savefig(join("trained_models_pics/MulticlassOpt", model.cnn_name + '_loss.png'), format = 'png')

    dict_path = join("trained_models/MulticlassOpt", model.cnn_name + "_state.pt")

    torch.save(model.state_dict(), dict_path)

    model.save_cnn_params()

    model.print_results_label(train_dataset, validation_dataset, label, 0.5)
    
    cm_05 = confusion_matrix(model, validation_dataset, label, 0.5)

    save_confusion_matrix(cm_05, label, '0.5', 'trained_models/MulticlassOpt')
    
    best_f1, best_t = get_best_threshold(model, validation_dataset, label)
    
    print('label{} F1: {}'.format(label,best_f1))
    
    best_thresholds.append(best_t)
    
    cm_bt = confusion_matrix(model, validation_dataset, label, 0.5)

    save_confusion_matrix(cm_bt, label, 'best_t', 'trained_models/MulticlassOpt')
    
with open('trained_models/MulticlassOpt/best_thresholds.pkl','wb') as f:

    pickle.dump(best_thresholds, f)

## Results

This first block of code will show us the how the loss develops during the training.

In [ ]:
plt.plot(epochs, train_losses, label = "train")

plt.plot(epochs, validation_losses, label = "validation")

plt.legend()

plt.title("loss vs epoch")

plt.savefig(join("trained_models_pics", model.cnn_name + '_loss.png'), format = 'png')

plt.show()

We save the model so that we can use it further in the future.

In [ ]:
dict_path = join("trained_models", model.cnn_name + "_state.pt")

torch.save(model.state_dict(), dict_path)

model.save_cnn_params()

And finally, we print the metrics to evaluate the model.

In [ ]:
model.print_results_label(train_dataset, test_dataset, label, 0.5)

In [ ]:
cm = confusion_matrix(model, test_dataset, label, 0.6)

In [ ]:
save_confusion_matrix(cm, label, 'trained_models/Multiclass')